In [1]:
import json
import os
import sys
import time

In [2]:
from icecream import ic
import numpy as np
import pandas as pd
from pprint import pprint

import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet

from tqdm import tqdm

In [3]:
os.chdir('../')
sys.path.append('../dataset/')
import coco_data_prep
import coco_api_helper
import config_dataset

loading annotations into memory...
Done (t=12.91s)
creating index...
index created!
loading annotations into memory...
Done (t=5.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [4]:
annot_filepaths = {'train': config_dataset.train_annotation_filepath,
                 'validation' : config_dataset.valid_annotation_filepath,
                 'test': config_dataset.test_annotation_filepath}

with open(annot_filepaths['train'], 'r') as f:
    train_annot = json.load(f)

### Create the Cateogries Label file

In [5]:
# cat_ids = [annot['id'] for annot in train_annot['categories']]
# cat_names = [annot['name'] for annot in train_annot['categories']]

# cat_table = pd.DataFrame(pd.Series(cat_ids, cat_names))
# cat_table.reset_index(drop=False, inplace=True)
# cat_table.rename(columns={
#     'index': 0,
#     0: 1
# }, inplace=True)
# cat_table = cat_table[[1, 0]]
# cat_table.to_csv('../dataset/coco_labels.txt', header=None)

### Helper Functions

In [6]:
def calc_new_dim(orig_h_or_w: int, padding, kernel, stride) -> int:
    """
    :orig_h_or_w: original width or original height of the input image
    given the original width or height, kernel size, pad width, stride size
    calculate the new width or height
    """
    new_dim = (orig_h_or_w + (2 * padding) - kernel) // stride + 1
    return new_dim

### Global Variables

In [7]:
with open('../dataset/categories.json', 'r') as j:
    categories = json.load(j)    

# catid_to_catname_df = pd.DataFrame.from_records(pd.Series([c.values() for c in categories])).drop(0, axis=1)
# catid_to_catname_df.to_csv('../dataset/coco_labels.txt', header=None, sep=',', mode='a')

In [8]:
train_np_data_dir = '../data/numpy_imgs/train_subset/'
train_jpg_data_dir = '../data/raw/train/train2014/'
train_annot_filepath = '../data/raw/train/annotations/instances_train2014.json'

with open('../dataset/imgs_by_supercategory.json', 'r') as f:
    desired_categories = json.load(f)

In [9]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

### Load Data : Train & Validation into DataLoader

In [11]:
# torch.cuda.empty_cache()

In [12]:
%load_ext autoreload

In [13]:
%autoreload

In [14]:
train_ds = coco_data_prep.COCODataset('train',
                                      train_np_data_dir, 
                                      train_annot_filepath,
                                      coco_data_prep.COCOAnnotationTransform(),
                                      sample_ratio=0.05,
                                      device='cpu')

loading annotations into memory...
Done (t=11.84s)
creating index...
index created!


100%|███████████████████████████████████████████| 82783/82783 [00:48<00:00, 1724.05it/s]


In [15]:
train_dl = coco_data_prep.get_dataloader(train_ds, 
                                         batch_size=512, 
                                         device='cpu')

In [16]:
# for i, (batch) in tqdm(enumerate(train_dl)):
#     image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]

#     image_batch = torch.stack(image_batch).to('cpu')
#     label_batch = torch.FloatTensor([l[0] for l in label_batch])
    
#     print(image_batch.shape)
#     print(label_batch.shape)
    

In [17]:
# val_ds = coco_data_prep.COCODataset('val',
#                                     val_np_data_dir, 
#                                     val_annot_filepath,
#                                     coco_data_prep.COCOAnnotationTransform(),
#                                     sample_ratio = 0.05,
#                                     device='cpu')

In [18]:
# val_dl = coco_data_prep.get_dataloader(val_ds, 
#                                      batch_size=512, 
#                                      device='cpu')

### Load Model

#### Densenet121

In [19]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

In [20]:
# model

### Build From Scratch - Densenet Inspired

In [21]:
class DensenetInspired(nn.Module):
    def __init__(self):
        super(DensenetInspired, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu1 = nn.ReLU()
        self.mp1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        
        # This part is Densenet inspired...recycle use of BN, RELU, CONV repeatedly
        self.bn2 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        
        self.bn3 = nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu3= nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        
        self.flat = nn.Flatten()
        self.lin1 = nn.LazyLinear(out_features=1024)
        self.fc1 = nn.Linear(1024, 80)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.mp1(out)
        
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)
        
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.conv3(out)
        
        out = self.flat(out)
        out = self.lin1(out)
        out = self.fc1(out)

        return out

#### Modeling Variables

In [22]:
model = DensenetInspired()

/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [23]:
optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)

optimizer = torch.optim.Adam(model.parameters(), **optim_params)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [24]:
criterion = nn.CrossEntropyLoss()

In [25]:
# from torchsummary import summary
# summary(model.cuda(), (3, 224, 224))

In [26]:
# def train_n_val ():
    
#     transform_test = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])
    
#     train_ds = coco_data_prep.COCODataset('train',
#                                           train_np_data_dir, 
#                                           train_annot_filepath,
#                                           coco_data_prep.COCOAnnotationTransform(),
#                                           sample_ratio=0.05,
#                                           device='cpu')
    
#     train_dl = coco_data_prep.get_dataloader(train_ds, 
#                                              batch_size=512, 
#                                              device='cpu')
    
#     val_ds = coco_data_prep.COCODataset('val',
#                                         val_np_data_dir, 
#                                         val_annot_filepath,
#                                         coco_data_prep.COCOAnnotationTransform(),
#                                         sample_ratio = 0.05,
#                                         device='cpu')
    
#     val_dl = 

In [27]:
torch.cuda.empty_cache()

In [28]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

In [ ]:
start = time.time()
for epoch in range(25):
    iter_time = AverageMeter()
    losses = AverageMeter()
    running_loss = 0.0
    count = 0
    
    for i, (batch) in tqdm(enumerate(train_dl)):
        image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
        image_batch = torch.stack(image_batch).to('cpu')
        label_batch = torch.LongTensor([l[0] for l in label_batch])
        
        optimizer.zero_grad()
        outputs = model(image_batch)

        loss = criterion(outputs, label_batch)
        print(loss)

        loss.backward()
        optimizer.step()
        count += 1

        losses.update(loss, outputs.shape[0])
        running_loss += loss

        iter_time.update(time.time() - start)
        if i % 10 == 0:
            print(('Epoch: [{0}][{1}/{2}]\t'
                   'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t'
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t')
                   .format(epoch, i, len(train_dl), iter_time=iter_time, loss=losses))

    print(f'Epoch-{epoch} lr: {optimizer.param_groups[0]["lr"]} running_loss: {running_loss}')
    

0it [00:00, ?it/s]

tensor(171.7637, grad_fn=<NllLossBackward>)


1it [00:07,  7.52s/it]

Epoch: [0][0/9]	Time 11.877 (11.877)	Loss 171.7637 (171.7637)	
tensor(116.1655, grad_fn=<NllLossBackward>)


2it [00:12,  5.91s/it]

...

Epoch-0 lr: 0.001 running_loss: 1086.35498046875



0it [00:00, ?it/s]

tensor(54.6155, grad_fn=<NllLossBackward>)


1it [00:08,  8.04s/it]

Epoch: [1][0/9]	Time 69.327 (69.327)	Loss 54.6155 (54.6155)	
tensor(73.0259, grad_fn=<NllLossBackward>)


In [ ]:
features_list = []
labels_list = []

# Use GPUs to speed up the inference, this should take around 10 minutes

model.to('cpu')
for batch in tqdm(train_dl):
    image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
    image_batch = torch.stack(image_batch).to('cpu') 

    with torch.no_grad():
        features_batch = model(image_batch) # .flatten(start_dim=1)
    features_list.append(features_batch)
    labels_list.extend(label_batch)